## Beer Recommendation System

1. **Task A** -> Scraping
2. **Task B** -> Bag-of-Words (TF-IDF + Sentiment)  
3. **Task C** -> Word2Vec Embeddings  
4. **Task D** -> SpaCy Embeddings  

In [ ]:
# # ===============================
# # Install Packages
# # ===============================
# !pip install beautifulsoup4 requests pandas

# # ===============================
# # Scraper
# # ===============================

# import time
# import random
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# import os
# import tempfile
# import shutil

# # -----------------------------
# # Step 1: Get top beers
# # -----------------------------
# def get_top_beer_urls():
#     url = 'https://www.beeradvocate.com/beer/top-rated/'
#     headers = {'User-Agent': 'Mozilla/5.0'}
#     try:
#         resp = requests.get(url, headers=headers, timeout=10)
#         resp.raise_for_status()
#     except Exception as e:
#         print(f"Failed to fetch top beers: {e}")
#         return []

#     soup = BeautifulSoup(resp.text, 'html.parser')
#     beer_data = []

#     for row in soup.select('tr'):
#         a_tag = row.find('a', href=lambda h: h and "/beer/profile/" in h)
#         if a_tag:
#             beer_name = a_tag.get_text(strip=True)
#             beer_url = 'https://www.beeradvocate.com' + a_tag['href']
#             beer_data.append({'name': beer_name, 'url': beer_url})

#     print(f"Found {len(beer_data)} beers")
#     return beer_data

# # -----------------------------
# # Human-like behavior (simplified for requests)
# # -----------------------------
# def human_delay(base_range=(2, 5)):
#     delay = random.uniform(*base_range)
#     print(f"Sleeping {delay:.1f}s")
#     time.sleep(delay)

# # -----------------------------
# # Scrape reviews using requests and BeautifulSoup
# # -----------------------------
# def scrape_all_reviews(beer_name, beer_url, max_reviews=40):
#     all_reviews = []
#     reviews_count = 0
#     current_url = beer_url
#     # Added more headers to mimic a browser
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
#         'Accept-Language': 'en-US,en;q=0.9',
#         'Referer': 'https://www.beeradvocate.com/beer/top-rated/', # Refer from the top-rated page
#         'DNT': '1', # Do Not Track Request Header
#         'Upgrade-Insecure-Requests': '1',
#     }


#     while True:
#         if reviews_count >= max_reviews:
#             print(f"Reached {max_reviews} reviews for {beer_name}.")
#             break

#         try:
#             resp = requests.get(current_url, headers=headers, timeout=15) # Increased timeout for review pages
#             resp.raise_for_status()
#         except Exception as e:
#             # print(f"Failed to fetch review page for {beer_name} ({current_url}): {e}")
#             break

#         soup = BeautifulSoup(resp.text, 'html.parser')
#         review_divs = soup.select('div[id^="rating_fullview_content_"]')

#         if not review_divs:
#             print(f"No more reviews found on this page for {beer_name}.")
#             break

#         for div in review_divs:
#             if reviews_count >= max_reviews:
#                 break
#             try:
#                 rating_el = div.select_one('.BAscore_norm')
#                 rating = rating_el.get_text(strip=True) if rating_el else "N/A"
#                 review_text_div = div.select_one('div[style*="line-height:1.4"]')
#                 full_text = review_text_div.decode_contents() if review_text_div else "" # Use decode_contents() to get inner HTML
#                 clean_review = (
#                     full_text.split('<br/><br/>Review date:')[0] # Adjusted split for BeautifulSoup output
#                              .split('<br/><br/>Bottle date:')[0] # Adjusted split for BeautifulSoup output
#                              .replace('<br/>', '\n') # Adjusted replace for BeautifulSoup output
#                              .strip()
#                 )
#                 if clean_review:
#                     all_reviews.append({'review': clean_review, 'rating': rating})
#                     reviews_count += 1
#             except Exception as e:
#                 print(f"Error extracting review for {beer_name}: {e}")
#                 continue

#         print(f"Scraped {len(review_divs)} reviews from this page, total so far: {reviews_count}")

#         # Find the next page link
#         next_link = soup.select_one('a.next')
#         if next_link and 'href' in next_link.attrs:
#             current_url = 'https://www.beeradvocate.com' + next_link['href']
#             human_delay((3, 7)) # Add delay between pages
#         else:
#             print(f"No more pages for {beer_name}.")
#             break

#     return all_reviews

# # ---------- Main workflow ----------
# def main():
#     csv_path = 'beer_reviews.csv'
#     MAX_TOTAL_REVIEWS = 15000

#     all_reviews_data = []
#     scraped_beers = set()

#     if os.path.exists(csv_path):
#         try:
#             df_existing = pd.read_csv(csv_path)
#             all_reviews_data = df_existing.to_dict("records")
#             scraped_beers = set(df_existing["product_name"].unique())
#             print(f"Resuming: {len(scraped_beers)} beers scraped, {len(all_reviews_data)} reviews.")
#         except pd.errors.EmptyDataError:
#             print(f"{csv_path} exists but is empty. Starting fresh.")
#             all_reviews_data = []
#             scraped_beers = set()
#         except Exception as e:
#             print(f"Error reading existing CSV file: {e}. Starting fresh.")
#             all_reviews_data = []
#             scraped_beers = set()

#     if not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0:
#          # Ensure header is written if starting fresh or file was empty
#         pd.DataFrame(columns=['review', 'rating', 'product_name']).to_csv(csv_path, index=False)


#     beer_list = get_top_beer_urls()

#     for i, beer in enumerate(beer_list):
#         if len(all_reviews_data) >= MAX_TOTAL_REVIEWS:
#             print(f"Reached {MAX_TOTAL_REVIEWS} total reviews. Stopping.")
#             break

#         beer_name = beer['name']
#         beer_url = beer['url']

#         if beer_name in scraped_beers:
#             print(f"Skipping {beer_name}, already scraped.")
#             continue

#         print(f"\nScraping beer {i+1}/{len(beer_list)}: {beer_name}")
#         # Call the refactored scraping function
#         revs = scrape_all_reviews(beer_name, beer_url, max_reviews=40)

#         for r in revs:
#             r['product_name'] = beer_name
#         all_reviews_data.extend(revs)

#         pd.DataFrame(all_reviews_data).to_csv(csv_path, index=False)
#         # print(f"Progress saved. Total reviews: {len(all_reviews_data)}")

#         human_delay((60, 180)) # Long break between beers

#     print(f"\nTotal reviews collected: {len(all_reviews_data)}")


# if __name__ == "__main__":
#     main()

Resuming: 230 beers scraped, 10000 reviews.
Found 250 beers
Skipping Kentucky Brunch Brand Stout, already scraped.
Skipping Marshmallow Handjee, already scraped.
Skipping Abraxas - Barrel-Aged, already scraped.
Skipping R&D Sour Fruit (Very Sour Blackberry), already scraped.
Skipping Hunahpu's Imperial Stout - Double Barrel Aged, already scraped.
Skipping Heady Topper, already scraped.
Skipping Mornin' Delight, already scraped.
Skipping King Julius, already scraped.
Skipping Pliny The Younger, already scraped.
Skipping King JJJuliusss, already scraped.
Skipping O.W.K., already scraped.
Skipping Blessed, already scraped.
Skipping M.J.K., already scraped.
Skipping Black Magick - Pappy Van Winkle, already scraped.
Skipping Fundamental Observation, already scraped.
Skipping Very Hazy, already scraped.
Skipping Zenne Y Frontera, already scraped.
Skipping Bourbon County Brand Vanilla Rye Stout, already scraped.
Skipping A Deal With The Devil - Triple Oak-Aged, already scraped.
Skipping Barre


## Task B - 1st part:
To determine the attributes, we first create a word frequency file and then pick the top three based on manual review of top 150-200 words sorted by descending occurences

In [11]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 47.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!pip install gensim

In [1]:
import pandas as pd
import re
import string
from collections import Counter
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already
nltk.download("stopwords")

df = pd.read_csv("/content/beer_reviews.csv")

# English stopwords
stop_words = set(stopwords.words("english"))

# Clean and tokenize reviews
def preprocess(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize
    words = re.findall(r"\b\w+\b", text)
    # Remove stopwords and short tokens
    words = [w for w in words if w not in stop_words and len(w) > 2]
    return words

# Apply preprocessing
all_words = []
for review in df["product_review"].dropna():
    all_words.extend(preprocess(review))

# Count frequencies
word_freq = Counter(all_words)

# Convert to DataFrame
freq_df = pd.DataFrame(word_freq.items(), columns=["word", "count"]).sort_values(by="count", ascending=False)

# Save to CSV
freq_df.to_csv("word_frequencies.csv", index=False)

print("Top 20 words:")
print(freq_df.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Top 20 words:
           word  count
23         beer   6869
15         head   5024
165        dark   4124
90    chocolate   4046
124       sweet   3342
77         like   3290
51        taste   3261
129     vanilla   3064
104         one   3017
35        notes   2968
34       coffee   2896
219        nice   2888
121     bourbon   2872
0          good   2679
8         light   2669
28         nose   2555
52         well   2479
2594     orange   2382
411       fruit   2319
44          bit   2311


## The top 3 attributes are **Sweet**, **Smooth** and **Hoppy**

##Task B - 2nd Part - Bag-of-Words (TF-IDF + Sentiment):

1. We have used a **TF-IDF vectorizer** combined with **cosine similarity** and
sentiment scores to recommend beers that best match user preferences.
2. The idea is to recommend beers based on how closely their reviews match the user’s preferred attributes (like sweet, smooth, hoppy), while also considering whether the reviews are written in a positive or negative tone


In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# ============================================
# Step 2: Data Preprocessing (Enhanced)
# ============================================

import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

# Example stop words
STOP_WORDS = set(["the", "and", "a", "of", "to", "in", "is", "it", "this"])

# Customer-specified attributes
USER_ATTRIBUTES = ["sweet", "smooth", "hoppy"]

# Attribute synonyms for BoW/TF-IDF models
ATTRIBUTE_SYNONYMS = {
    "sweet": ["sweet", "sugary", "caramel", "vanilla", "molasses", "toffee",
              "chocolate", "bourbon", "maple"],
    "smooth": ["smooth", "silky", "creamy", "mellow", "rounded", "velvet"],
    "hoppy": ["hop", "hops", "hoppy", "bitter", "bitterness", "citrus",
              "pine", "resin", "floral", "grapefruit", "orange"]
}

ATTRIBUTE_RULES = {
    "sweet": {
        "neg_strong": [r"\bcloying\b", r"\btoo\s+sweet\b", r"\boverly\s+sweet\b",
                       r"\bsickly\s+sweet\b", r"\bexcessively\s+sweet\b"],
        "neg": [r"\bnot\s+sweet\b", r"\bartificial(ly)?\s+sweet\b", r"\bsyrupy\b",
                r"\blacks?\s+sweet(ness)?\b"],
        "pos": [r"\bpleasantly\s+sweet\b", r"\bsubtly\s+sweet\b",
                r"\bbalanced\s+sweet(ness)?\b"]
    },
    "smooth": {
        "neg_strong": [r"\bnot\s+smooth\b", r"\bharsh\b", r"\bastringent\b", r"\brough\b"],
        "neg": [r"\bthin\b", r"\bsharp\b"],
        "pos": [r"\bsilky\b", r"\bcreamy\b", r"\bvelvety\b", r"\bbuttery\b",
                r"\b(very|super|ultra|really|exceptionally)\s+smooth\b",
                r"\bsmooth\s+and\s+(silky|creamy|velvety|buttery)\b"]
    },
    "hoppy": {
        "neg_strong": [r"\boverly\s+hoppy\b", r"\btoo\s+hoppy\b", r"\bexcessively\s+hoppy\b"],
        "neg": [r"\bnot\s+hoppy\b", r"\blacks?\s+hops?\b"],
        "pos": [r"\bnice\s+hoppy\b", r"\bpleasantly\s+hoppy\b", r"\bhoppy\s+bite\b",
                r"\bbalanced\s+hoppy\b", r"\brefreshingly\s+hoppy\b"]
    }
}

def preprocess_text(text: str) -> list:
    if not isinstance(text, str):
        return []
    text = re.sub(r"[^a-zA-Z]", " ", text).lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in STOP_WORDS and len(t) > 2]
    return tokens

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

def compute_attribute_sentiment(text: str) -> dict:
    """
    Compute attribute-specific sentiment based on ATTRIBUTE_RULES.
    Returns a dict: {attribute: score}, where score is in [-1, 1]
    """
    scores = {}
    for attr, rules in ATTRIBUTE_RULES.items():
        score = 0
        for pattern in rules.get("neg_strong", []):
            if re.search(pattern, text):
                score -= 1.0
        for pattern in rules.get("neg", []):
            if re.search(pattern, text):
                score -= 0.5
        for pattern in rules.get("pos", []):
            if re.search(pattern, text):
                score += 0.5
        # Clip score between -1 and 1
        scores[attr] = max(min(score, 1.0), -1.0)
    return scores

def load_and_prepare(path: str) -> pd.DataFrame:
    """
    Load reviews, clean, tokenize, compute VADER sentiment, and attribute-specific sentiment.
    """
    df = pd.read_csv(path)
    if "product_review" not in df.columns or "product_name" not in df.columns:
        raise ValueError("CSV must contain 'product_name' and 'product_review' columns.")

    # Normalize + tokenize
    df["clean_review"] = df["product_review"].apply(normalize_text)
    df["tokens"] = df["product_review"].apply(preprocess_text)

    # Sentiment (VADER)
    sia = SentimentIntensityAnalyzer()
    df["sentiment_raw"] = df["product_review"].apply(
        lambda x: sia.polarity_scores(str(x))["compound"]
    )

    # Attribute-specific sentiment
    df["attribute_sentiment"] = df["product_review"].apply(compute_attribute_sentiment)

    print(f"✅ Loaded {len(df)} reviews after cleaning and sentiment computation.")
    return df.dropna(subset=["clean_review"]).reset_index(drop=True)


In [4]:
# ============================================
# Step 3: Bag-of-Words Model (Task B)
# ============================================

def recommend_bow(df: pd.DataFrame, attributes: list, alpha=0.7, top_n=23):
    """
    TF-IDF + Cosine Similarity + Sentiment
    """
    grouped = df.groupby("product_name").agg({
        "clean_review": " ".join,
        "sentiment_raw": "mean"
    }).reset_index()

    # Attribute synonyms for BoW/TF-IDF models
    ATTRIBUTE_SYNONYMS = {
    "sweet": ["sweet", "sugary", "caramel", "vanilla", "molasses", "toffee",
              "chocolate", "bourbon", "maple"],
    "smooth": ["smooth", "silky", "creamy", "mellow", "rounded", "velvet"],
    "hoppy": ["hop", "hops", "hoppy", "bitter", "bitterness", "citrus",
              "pine", "resin", "floral", "grapefruit", "orange"]
    }

    # Build query from attributes + synonyms
    query_terms = []
    for a in attributes:
        query_terms.extend(ATTRIBUTE_SYNONYMS.get(a, [a]) * 3)
    query_doc = " ".join(query_terms)

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(stop_words="english", max_features=30000)
    tfidf_matrix = vectorizer.fit_transform(grouped["clean_review"])
    query_vec = vectorizer.transform([query_doc])

    # Cosine similarity
    cos_sims = cosine_similarity(query_vec, tfidf_matrix).flatten()
    cos_norm = MinMaxScaler().fit_transform(cos_sims.reshape(-1, 1)).ravel()

    # Sentiment normalization
    sent_norm = MinMaxScaler().fit_transform(grouped[["sentiment_raw"]]).ravel()

    # Final score
    final_score = alpha * cos_norm + (1 - alpha) * sent_norm

    recs = grouped.copy()
    recs["cosine_sim"] = cos_sims
    recs["final_score"] = final_score
    return recs.sort_values("final_score", ascending=False).head(top_n)

## Task C: Using Pretrained Word Vectors

Here we use spaCy’s pretrained word vectors to capture the meaning of words in reviews.

1. First, we turn both the reviews and the chosen attributes (with their synonyms) into vector representations.

2. Then we measure cosine similarity between reviews and attributes to see how closely they align.

3. Finally, we mix in the sentiment score so products that are both relevant and positively reviewed get ranked higher.

This gives us a recommendation list that balances semantic closeness with customer sentiment

In [6]:
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [7]:
# ============================================
# Pretrained Word Vectors (Task C)
# ============================================

def recommend_vectors(df: pd.DataFrame, attributes: list, alpha=0.7, top_n=23):
    """
    spaCy pretrained vectors + cosine similarity + sentiment
    """
    try:
        nlp = spacy.load("en_core_web_md")
    except:
        raise RuntimeError("Please install en_core_web_md: python -m spacy download en_core_web_md")

    prod = df.groupby("product_name").agg({
        "product_review": " ".join,
        "sentiment_raw": "mean"
    }).reset_index()

    # Build query doc
    query_terms = " ".join([t for a in attributes for t in ATTRIBUTE_SYNONYMS.get(a, [a])])

    def doc_vector(text):
        doc = nlp(text)
        vecs = [t.vector for t in doc if t.has_vector and not t.is_stop]
        return np.mean(vecs, axis=0) if vecs else np.zeros(nlp.vocab.vectors_length)

    prod_vecs = np.vstack([doc_vector(t) for t in prod["product_review"]])
    query_vec = doc_vector(query_terms).reshape(1, -1)

    # Cosine similarity
    cos_sims = cosine_similarity(prod_vecs, query_vec).flatten()
    cos_norm = MinMaxScaler().fit_transform(cos_sims.reshape(-1, 1)).ravel()
    sent_norm = MinMaxScaler().fit_transform(prod[["sentiment_raw"]]).ravel()

    final_score = alpha * cos_norm + (1 - alpha) * sent_norm

    recs = prod.copy()
    recs["cosine_sim"] = cos_sims
    recs["final_score"] = final_score
    return recs.sort_values("final_score", ascending=False).head(top_n)

## Task D: Training Custom Word2Vec Embeddings

In this step, instead of relying on pretrained vectors, we train our own Word2Vec model directly on the beer reviews.

1. Each word in the reviews is represented as a vector, but the embeddings are now tailored to the beer domain rather than general English.

2. We build product-level vectors by averaging the embeddings of words from their reviews.

3. Then we compare these product vectors with the attribute vectors using cosine similarity, and again combine that with sentiment scores.

4. This approach lets us capture domain-specific nuances (e.g., words like “hoppy”, “malty”, “crisp” may get richer meaning) which general-purpose vectors might miss.

In [8]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
# ============================================
# Step 5: Custom Word2Vec (Task D)
# ============================================

from gensim.models import Word2Vec

def train_custom_embeddings(tokenized_reviews, vector_size=100, window=5, min_count=5):
    """
    Train custom Word2Vec embeddings on beer reviews.
    """
    model = Word2Vec(sentences=tokenized_reviews,
                     vector_size=vector_size,
                     window=window,
                     min_count=min_count,
                     workers=4)
    return model


def recommend_custom(df: pd.DataFrame, attributes: list, model: Word2Vec, alpha=0.7, top_n=23):
    """
    Custom Word2Vec + cosine similarity + sentiment
    """
    # Attribute vector
    query_vecs = [model.wv[w] for w in attributes if w in model.wv]
    if not query_vecs:
        print("⚠️ No attribute words found in custom vocab.")
        return pd.DataFrame()
    query_vector = np.mean(query_vecs, axis=0)

    # Product vectors
    product_vectors = {}
    for name, group in df.groupby("product_name"):
        review_vecs = []
        for tokens in group["tokens"]:
            word_vecs = [model.wv[w] for w in tokens if w in model.wv]
            if word_vecs:
                review_vecs.append(np.mean(word_vecs, axis=0))
        if review_vecs:
            product_vectors[name] = np.mean(review_vecs, axis=0)

    # Cosine similarity
    similarities = {}
    for name, vec in product_vectors.items():
        sim = np.dot(vec, query_vector) / (np.linalg.norm(vec) * np.linalg.norm(query_vector))
        similarities[name] = sim

    recs = pd.DataFrame(similarities.items(), columns=["product_name", "cosine_sim"])

    # Sentiment
    sentiments = df.groupby("product_name")["sentiment_raw"].mean().reset_index()
    recs = recs.merge(sentiments, on="product_name", how="left")

    # Normalize + final score
    recs["cos_norm"] = MinMaxScaler().fit_transform(recs[["cosine_sim"]])
    recs["sent_norm"] = MinMaxScaler().fit_transform(recs[["sentiment_raw"]])
    recs["final_score"] = alpha * recs["cos_norm"] + (1 - alpha) * recs["sent_norm"]

    return recs.sort_values("final_score", ascending=False).head(top_n)

In [13]:
# ----------------------------
# Execute all 3 methods
# ----------------------------
import nltk
nltk.download('punkt_tab')
nltk.download('vader_lexicon')
import spacy
nlp = spacy.load("en_core_web_md")
# pd.set_option('display.max_colwidth', None)

if __name__ == "__main__":
    df = load_and_prepare("/content/beer_reviews.csv")

    # --- Task B ---
    print("\n=== Task B: Bag-of-Words Recommendations ===")
    bow_recs = recommend_bow(df, USER_ATTRIBUTES).head(3)
    display(bow_recs)   # Shows as table in Colab

    # --- Task C ---
    print("\n=== Task C: Pretrained Word Vectors (spaCy) ===")
    vec_recs = recommend_vectors(df, USER_ATTRIBUTES).head(3)
    display(vec_recs)

    # --- Task D ---
    print("\n=== Task D: Custom Word2Vec Embeddings ===")
    custom_model = train_custom_embeddings(df["tokens"].tolist())
    custom_recs = recommend_custom(df, USER_ATTRIBUTES, custom_model).head(3)
    display(custom_recs)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


✅ Loaded 10000 reviews after cleaning and sentiment computation.

=== Task B: Bag-of-Words Recommendations ===


,product_name,clean_review,sentiment_raw,cosine_sim,final_score
166,Permanent Funeral,drunkin fruits and dripping resinous hops fuck...,0.701922,0.321052,0.892267
45,CBS (Canadian Breakfast Stout),copenhagen 30 4 2020 35 5 cl bottle from meny ...,0.754722,0.307244,0.892241
47,Canuckley,pours a dense dark black with short lived dark...,0.719647,0.303511,0.863611



=== Task C: Pretrained Word Vectors (spaCy) ===


,product_name,product_review,sentiment_raw,cosine_sim,final_score
150,Moment Of Clarity,Black coal with a nice tan thick creamy head. ...,0.808061,0.797539,0.953826
25,BVC,Blend 2 at Side Project Cellar 3/12/25. I did...,0.887675,0.786592,0.943723
14,All That Is And All That Ever Will Be,"Pours a dark brown, but appears black in the g...",0.743565,0.793231,0.894274



=== Task D: Custom Word2Vec Embeddings ===


,product_name,cosine_sim,sentiment_raw,cos_norm,sent_norm,final_score
166,Permanent Funeral,0.738280,0.701922,1.000000,0.640888,0.892267
73,Double Dry Hopped Double Mosaic Daydream,0.727004,0.722202,0.975784,0.680097,0.887078
121,Jjjuliusss,0.684755,0.774121,0.885057,0.780470,0.853681


# Tfidf vs. SpaCy

The two recommendation methods were applied to a collection of user beer reviews, yielding the following top 3 results based on their respective scoring metrics.

## Top 3 Beer Recommendations by Method

| Method | Rank 1 Beer | Rank 2 Beer | Rank 3 Beer | Primary Style |
| :--- | :--- | :--- | :--- | :--- |
| **Tfidf (Bag of Words)** | Permanent Funeral | CBS (Canadian Breakfast Stout) | Canuckley | Mixed (Imperial IPA & Stouts) |
| **SpaCy (Semantic)** | Moment Of Clarity | BVC | All That Is And All That Ever Will Be | Adjunct Stouts |

---

## Which Method Gives Better Recommendations?

The **SpaCy** method appears to provide **better and more coherent recommendations** in this specific context.

### Reasoning

1.  **Semantic Coherence (SpaCy)**:
    * SpaCy's top recommendations (**Moment Of Clarity**, **BVC**, and **All That Is And All That Ever Will Be**) are all high-rated, **dessert-style Imperial Stouts/Milk Stouts** that prominently feature adjuncts like **chocolate, coffee, maple syrup, and vanilla**.
    * This suggests that the model effectively used **semantic processing** to understand the specific flavor profile being described (i.e., sweet, adjunct-heavy, pastry stout characteristics) and matched it to a highly similar group of beers.

2.  **Keyword Focus (Tfidf)**:
    * The Tfidf model's list is inconsistent, mixing the **Imperial IPA Permanent Funeral** with the **Imperial Stouts CBS and Canuckley**.
    * **Tfidf** relies on the frequency and distinctiveness of individual words (term frequency-inverse document frequency). It likely scored both beer styles highly based on **generic positive descriptors** (e.g., "great," "smooth," "complex," "winner") rather than the unique flavor terms.
    * A simple count of terms like "pine," "citrus," and "bitter" (IPA) would be less semantically related to "chocolate," "maple," and "bourbon" (Stouts), indicating that Tfidf failed to capture the distinct *meaning* of the reviews and group by flavor profile.

For recommending a product like beer, where the **flavor profile and style are critical**, a model with **semantic understanding (SpaCy)** is more valuable than a basic **word-frequency model (Tfidf)**, as it correctly groups products with similar taste and style characteristics.

## Task E: Comparing with Top-Rated Products  

In this task, we ask: *What if we ignore similarity and sentiment scores, and simply choose the 3 highest-rated products from the dataset?*  

Key questions to consider:  
- Would these products actually meet the needs of a user looking for personalized recommendations?  
- Why or why not?  
- Let’s analyze and justify whether relying on raw ratings alone is a good strategy.  


In [14]:
import pandas as pd

# Step 1: Find max rating
max_rating = df['user_rating'].max()

# Step 2: Filter beers with max rating
top_rated = df[df['user_rating'] == max_rating]

# Step 3: Count frequency
beer_freq = top_rated['product_name'].value_counts()

# Step 4: Pick top 3
top_3_beers = beer_freq.head(3)
print(top_3_beers)

product_name
Kentucky Brunch Brand Stout       25
Trappist Westvleteren 12 (XII)    24
Heady Topper                      16
Name: count, dtype: int64


# Beer Recommendation Analysis

## Ignoring Similarity and Sentiment

If we ignore similarity and sentiment, and **only use ratings**, the top-3 beers are:

| Rank | Beer                          | Rating | Reviews |
|------|-------------------------------|--------|---------|
| 1    | Kentucky Brunch Brand Stout   | 5.0    | 25      |
| 2    | Trappist Westvleteren 12 (XII)| 5.0    | 24      |
| 3    | Heady Topper                  | 5.0    | 16      |

These come from the **global highest-rated set**, regardless of the user's requested attributes.

---

## User Attributes Chosen
- **Sweet**
- **Smooth**
- **Hoppy**

---

## Attribute-Aware Models (Tasks B–D)

When using **BoW (Task B)**, **Pretrained Vectors (Task C)**, and **Custom Embeddings (Task D)**, the following beers consistently appeared at the top:

- Permanent Funeral  
- Pliny the Elder  
- Triple Sunshine  
- Abrasive Ale  
- Double Dry Hopped Mosaic Daydream  

These beers are described in reviews with strong **hoppy, citrus, bitter, or smooth** flavor notes — i.e., directly relevant to the user's preferences.

---

## Why Ratings Alone Fail

- **Kentucky Brunch Brand Stout** and **Trappist Westvleteren 12** are world-class imperial stouts.  
  - Reviews emphasize *richness, hype, and exclusivity*.  
  - They are highly rated but **not hoppy or citrusy**.  
- **Heady Topper** is different: as a Double IPA, it **does align** with *hoppy* and sometimes *smooth*.  
- Therefore, a **rating-only approach risks recommending beers that are excellent overall but irrelevant** to the chosen attributes.

---

## Role of Attribute + Sentiment Scoring

- Ratings capture **aggregate popularity**, not **personalized flavor alignment**.  
- Attribute + sentiment scoring ensures:  
  1. Beers are **similar to requested attributes** (hoppy, smooth, citrus).  
  2. Beers are **positively received in reviews**.  
- This prevents recommending beers that are “top-rated” but **off-profile**.

---

If we ignored similarity and sentiment and **only chose the three highest-rated beers**, we would recommend:

1. Kentucky Brunch Brand Stout  
2. Trappist Westvleteren 12 (XII)  
3. Heady Topper  

- These are **highly acclaimed**.  
- But **only Heady Topper** aligns with *hoppy, smooth*.  
- The two stouts do not match the flavor profile and would likely disappoint the user.  

**Conclusion:**  
Relying on ratings alone is insufficient — popularity ≠ attribute relevance.  
Our similarity + sentiment models (Tasks B–D) produced recommendations that consistently emphasize **hoppiness, citrus, and smoothness**, making them **better tailored** to the user's needs.


##  Task F: Selecting 10 Beers and Finding the Closest Match

In [15]:
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------
# Step 2. Pick 10 beers
# ---------------------------
sampled_beers = df['product_name'].drop_duplicates().sample(10)
subset_df = df[df['product_name'].isin(sampled_beers)]

# ---------------------------
# Step 3. Concatenate reviews per beer
# ---------------------------
beer_docs = subset_df.groupby('product_name')['product_review'].apply(
    lambda x: " ".join(str(r) for r in x)
).reset_index()

# ---------------------------
# Step 4. Load spaCy embeddings
# ---------------------------
nlp = spacy.load("en_core_web_md")

def get_embedding(text):
    doc = nlp(text)
    # Average word vectors (ignore stopwords & out-of-vocab tokens)
    vectors = [token.vector for token in doc if token.has_vector and not token.is_stop]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(nlp.vocab.vectors_length)

beer_docs['embedding'] = beer_docs['product_review'].apply(get_embedding)

# Convert to matrix
embedding_matrix = np.vstack(beer_docs['embedding'].values)

# ---------------------------
# Step 5. Cosine similarity
# ---------------------------
similarity_matrix = cosine_similarity(embedding_matrix)

# ---------------------------
# Step 6. Pick a target beer and find competitor
# ---------------------------
target_beer = beer_docs['product_name'].iloc[0]   # first beer in list
target_idx = 0

# Get similarity scores (exclude self)
similarities = similarity_matrix[target_idx]
similarities[target_idx] = -1  # so we don't pick itself

# Most similar beer
competitor_idx = np.argmax(similarities)
competitor_beer = beer_docs['product_name'].iloc[competitor_idx]

print(f"Target beer: {target_beer}")
print(f"Most similar competitor: {competitor_beer}")

Target beer: Blåbær Lambik
Most similar competitor: Westly


## Methodology

### 1. Selection of Beers  
- Ten distinct beers were randomly sampled from the dataset.  
- This ensures diversity across styles and brands, minimizing bias in the analysis.  


### 2. Aggregation of Reviews  
- For each beer, all individual reviews were **concatenated into a single document**.  
- This creates a comprehensive representation of customer sentiment and descriptive language.  
- Instead of analyzing reviews separately, the **collective voice of consumers** is treated as one profile per beer.  


### 3. Text Representation through Embeddings  
- Reviews were converted into numerical form using **spaCy’s `en_core_web_md` model**, which provides pretrained **300-dimensional word embeddings**.  
- For each beer:  
  - Stopwords and words without embeddings were excluded.  
  - Vectors for the remaining tokens were **averaged**, producing a single vector.  
- The result is a semantic vector capturing the **essence of how consumers describe the beer**.  


### 4. Measuring Similarity  
- To compare beers, **cosine similarity** was applied between their vectors.  
- A higher cosine similarity score means that consumer language describing two beers is more alike.  
- This indicates that the beers are **perceived as closer substitutes**.  


### 5. Identifying the Closest Competitor  
- For the **chosen target beer**, similarity scores were calculated against the other nine beers.  
- The beer with the **highest similarity score** (excluding the target itself) was identified as the **closest competitor**.  


## Logic Behind the Approach
- This method emphasizes **customer perception** rather than objective product attributes (e.g., ABV, style, brewery).  
- Consumers describe beers based on **taste, mouthfeel, bitterness, sweetness, aroma**, etc.  
- By embedding this descriptive language, hidden relationships between beers can be revealed.  
- Two beers may emerge as competitors even if they come from **different breweries or styles**, as long as consumers perceive them similarly.  


## Conclusion
By averaging **word embeddings** from reviews and applying **cosine similarity**, we identified the closest competitor to the target beer among a sample of ten.  
This demonstrates how **NLP can uncover consumer-driven insights** in markets where traditional categories may fail to capture true competitive dynamics.
